## PAIR PROGRAMMING LIMPIEZA III

### Valores Nulos Sklearn
---

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de **Limpieza III** donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers. Hoy volveremos a gestionar valores nulos, pero en este caso usaremos los métodos de imputación de sklearn.

Antes de seguir, recordamos las preguntas que nos planteamos al principio del pair programming de EDA para dirigir nuestro análisis.

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I y II ya dejamos algunas columnas limpitas:


- age ✔️ la limpiamos en el pair de Limpieza II
- species ✔️ la limpiamos en el pair de Limpieza II
- country ✔️ la limpiamos en el pair de Limpieza II
- fatal ✔️ la limpiamos en el pair de Pandas V
- year ✔️ es una columna de tipo float deberíamos convertirla a integer
- sex ✔️ la limpiamos en el pair de Pandas V
- fecha ✔️ la limpiamos en el pair de Pandas V

Es el momento de ponernos a trabajar con los valores nulos de nuevo 💪🏽. A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

In [2]:
df = pd.read_pickle("../files/attacks10.pkl")
df.head()

,case_number,year,type,country,activity,age,species_,date,mes_ataque,fatal,sexo,cat_species,edades,mes
0,1800.00.00,1997,Unprovoked,seychelles,a corsair's boat was overturned,NaN,NaN,1800,NaN,Y,F,NaN,NaN,NaN
1,1797.05.28.R,1997,Unprovoked,NaN,Dropped overboard,NaN,NaN,Reported May-28-1797,May,Y,NaN,NaN,NaN,May
2,1792.09.12,1997,Provoked,england,Fishing,NaN,NaN,12-Sep-1792,Sep,Y,M,NaN,NaN,Sep
3,1791.00.00,1997,Unprovoked,australia,NaN,NaN,NaN,1791,NaN,Y,F,NaN,NaN,NaN
4,1788.05.10,1997,Boat,australia,Fishing,NaN,NaN,10-May-1788,May,N,NaN,NaN,NaN,May


**EJERCICIO 1**

Es el momento de eliminar los nulos:

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [3]:
# Comprobamos el número de valores nulos de la columna 'edades'
df['edades'].isnull().sum() 

162

In [4]:
# Almacenamos en una variable todos los parámetros a aplicar por el método, en este caso cambiar los valores nulos por la media
imputer= SimpleImputer(strategy="mean",missing_values=np.nan) 

In [5]:
# Sobreescribimos y aplicamos el método .fit sobre la columna 'edades'
imputer= imputer.fit(df[['edades']]) 

In [6]:
# Sobreescribimos sobre la columna los cambios que aplicamos con el método .transform
df['edades'] = imputer.transform(df[['edades']]) 

In [7]:
# Comprobamos que no hay valores nulos
df['edades'].isnull().sum() 

0

---

- Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
    💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [8]:
# Comprobamos el número de valores nulos de la columna 'sexo'
df['sexo'].isnull().sum() 

14

In [9]:
# Almacenamos en una variable todos los parámetros que queremos aplicar, en este caso cambiar los valores nulos por la moda
imputer= SimpleImputer(strategy="most_frequent",missing_values=np.nan)

In [10]:
# Sobreescribimos y aplicamos el método .fit sobre la columna 'sexo'
imputer= imputer.fit(df[['sexo']])

In [11]:
# Sobreescribimos sobre la columna los cambios que aplicamos con el método .transform
df['sexo'] = imputer.transform(df[['sexo']])

In [12]:
# Comprobamos que no hay valores nulos
df['sexo'].isnull().sum()

0

---

- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [13]:
df['type'].isnull().sum() # Comprobamos los valores nulos de la columna 'type' y vemos que no hay ninguno

0

---

- Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [14]:
# Guardamos en la variable numericas la selección de las columnas numéricas del DataFrame(DF) con el método .selec_dtypes
numericas= df.select_dtypes(include=np.number)
numericas.head()

,year,edades
0,1997,27.878808
1,1997,27.878808
2,1997,27.878808
3,1997,27.878808
4,1997,27.878808


In [15]:
# Al imputar el método KNN le damos 5 vecinos de comparación y aplicamos el método .fit a las columnas arriba seleccionadas
imputerKNN = KNNImputer(n_neighbors=5)
imputerKNN.fit(numericas)

KNNImputer()

In [16]:
# Aplicamos el siguiente paso con el método .transfrom
numericas_knn= imputerKNN.transform(numericas)

In [17]:
# Creamos un DF con las columnas numéricas actualizadas
df_knn_imputer = pd.DataFrame(numericas_knn, columns = numericas.columns)


In [18]:
# Sacamos el nombre de las columnas del DF sin nulos
columnas_knn = df_knn_imputer.columns

In [19]:
# Eliminamos esas columnas del DF
df.drop(columnas_knn, axis = 1, inplace = True)

In [20]:
# Creamos nuevas columnas en el DF original con en el DF creado con las numericas_trans
df[columnas_knn] = numericas_knn

In [ ]:
# Comprobamos que no hay valores nulos en las nuevas columnas
df.isnull().sum()

---

- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [22]:
# Abrimos de nuevo archivo inicial y lo guardamos en otro DataFrame para volver a reemplazar los valores nulos
df_2 = pd.read_pickle("../files/attacks10.pickle")
df_2.head()

,case_number,year,type,country,activity,age,species_,date,mes_ataque,fatal,sexo,cat_species,edades,mes
0,1800.00.00,1997,Unprovoked,seychelles,a corsair's boat was overturned,NaN,NaN,1800,NaN,Y,F,NaN,NaN,NaN
1,1797.05.28.R,1997,Unprovoked,NaN,Dropped overboard,NaN,NaN,Reported May-28-1797,May,Y,NaN,NaN,NaN,May
2,1792.09.12,1997,Provoked,england,Fishing,NaN,NaN,12-Sep-1792,Sep,Y,M,NaN,NaN,Sep
3,1791.00.00,1997,Unprovoked,australia,NaN,NaN,NaN,1791,NaN,Y,F,NaN,NaN,NaN
4,1788.05.10,1997,Boat,australia,Fishing,NaN,NaN,10-May-1788,May,N,NaN,NaN,NaN,May


In [23]:
# Guardamos en la variable numericas2 la selección de las columnas numéricas del DataFrame(DF) con el método .selec_dtypes
numericas2= df_2.select_dtypes(include=np.number)
numericas2.head()

,year,edades
0,1997,NaN
1,1997,NaN
2,1997,NaN
3,1997,NaN
4,1997,NaN


In [25]:
# Comprobamos los valores nulos de las columnas numéricas
numericas2.isnull().sum()

year        0
edades    162
dtype: int64

In [26]:
# Almacenamos en una variable todos los parámetros que queremos aplicar, en este caso cambiar los valores nulos por la media, redondeando a 3 decimales,
# iterando 20 veces de manera ascendente.
imputer2 = IterativeImputer(max_iter=20, n_nearest_features=None, imputation_order='ascending', initial_strategy= 'median', tol=3)

In [27]:
# Lo aplicamos con el método .fit
imputer2.fit(numericas2) # Revisar porque no nos coge los parámetros que le estamos especificando y sigue con los parámtros por defecto.

IterativeImputer(initial_strategy='median', max_iter=20, tol=3)

In [28]:
#df_2['edades_mean']= imputer.transform(df_2['edades']) # Revisar este código que nos ha dicho Cesar

In [29]:
# Tranformamos los datos con el método .transform y convertimos el resultado el un DF
numericas_trans2 = pd.DataFrame(imputer2.transform(numericas2), columns = numericas2.columns)

In [30]:
# Sacamos el nombre de las columnas del DF sin nulos
columnas2 = numericas_trans2.columns

In [31]:
# Eliminamos esas columnas del DF
df_2.drop(columnas2, axis = 1, inplace = True)

In [32]:
# Creamos nuevas columnas en el DF original con en el DF creado con las numericas_trans2
df_2[columnas2] = numericas_trans2[columnas2]

In [33]:
# Comprobamos que no hay valores nulos en las nuevas columnas
df_2.isnull().sum()

case_number      0
type             0
country         10
activity        31
age            154
species_       126
date             0
mes_ataque     141
fatal           97
sexo            14
cat_species    126
mes            170
year             0
edades           0
dtype: int64

In [34]:
# Comprobamos los cambios finales en el DF 
df_2.head()  ##### revisar las celdas de arriba pues está utilizando los parámetros por defecto y no los dados

,case_number,type,country,activity,age,species_,date,mes_ataque,fatal,sexo,cat_species,mes,year,edades
0,1800.00.00,Unprovoked,seychelles,a corsair's boat was overturned,NaN,NaN,1800,NaN,Y,F,NaN,NaN,1997.0,28.421549
1,1797.05.28.R,Unprovoked,NaN,Dropped overboard,NaN,NaN,Reported May-28-1797,May,Y,NaN,NaN,May,1997.0,28.421549
2,1792.09.12,Provoked,england,Fishing,NaN,NaN,12-Sep-1792,Sep,Y,M,NaN,Sep,1997.0,28.421549
3,1791.00.00,Unprovoked,australia,NaN,NaN,NaN,1791,NaN,Y,F,NaN,NaN,1997.0,28.421549
4,1788.05.10,Boat,australia,Fishing,NaN,NaN,10-May-1788,May,N,NaN,NaN,May,1997.0,28.421549


---

- ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

    - Con el método de KNN Imputer no podemos asignarle parámetros para controlar el orden de comparación con las columnas, ni redondear decimales, ni especificar con que estadístico queremos reemplazar, cosa que con el método Iterative Imputer si podemos hacer.    
    - Nos da la impresión que según la cantidad de columnas numéricas(por ejemplo, si hay pocas) que el método KNN funciona con más precisión que el II.

---

EJERCICIO 2

Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

In [35]:
df.to_pickle('../files/attacks12.pkl')